### Scraping and parcing ISIN assigned registration numbers for 2024 year

In [48]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

We have 423 pages with news about ISIN numbers assigning

In [51]:
period = range(1,424)

In [52]:
parsed_results = []
for i in tqdm(period):
    url = f'https://www.isin.ru/ru/ru_isin/news_c/index.php?page22={i}'
    r = requests.get(url)
    if r.status_code == 200:
        soup = BeautifulSoup(r.content, 'xml')
        results = soup.find_all("div", class_="news_sep")
        for item in results:
            try:
                date = item.find('span', class_='date2 border_date2').text
                web_link = item.find('a', class_='grey')['href']
                text = item.find('a', class_='grey').text.strip()
                parsed_results.append({
                                        "date": date,
                                        "web-link": web_link,
                                        "text": text
                                      })
            except AttributeError as e:
                print(f"Skipping an item due to missing data: {e}")
    else:
        print(f'There is a problem with loading {i} page')
        continue

df = pd.DataFrame(parsed_results)
df

100%|██████████| 423/423 [03:38<00:00,  1.93it/s]


,date,web-link,text
0,28.12.24,?id22=667879,"О присвоении ISIN кода облигациям АО ""Сбербанк..."
1,28.12.24,?id22=667877,"О присвоении ISIN кода облигациям АО ""Сбербанк..."
2,28.12.24,?id22=667875,"О присвоении ISIN кода облигациям АО ""Сбербанк..."
3,28.12.24,?id22=667873,"О присвоении ISIN кода облигациям АО ""Сбербанк..."
4,28.12.24,?id22=667871,"О присвоении ISIN кода облигациям АО ""Сбербанк..."
...,...,...,...
8436,10.01.22,?id22=651006,О присвоении ISIN кода облигациям Банка ВТБ (П...
8437,10.01.22,?id22=651004,О присвоении ISIN кода облигациям Банка ВТБ (П...
8438,10.01.22,?id22=651002,О присвоении ISIN кода облигациям Банка ВТБ (П...
8439,10.01.22,?id22=651000,О присвоении ISIN кода облигациям Банка ВТБ (П...


In [ ]:
df.to_csv('isin_registration.csv')

Since we are interested only in bonds we can filter them:

In [56]:
contain = ['облигаци']
for c in contain:
    filtered = df[df['text'].str.contains(c, case=False, na=False)]

filtered.reset_index(inplace=True)
filtered.drop(columns='index', inplace=True)
filtered

C:\Users\lenovo\AppData\Local\Temp\ipykernel_6836\2558127847.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered.drop(columns='index', inplace=True)


,date,web-link,text
0,28.12.24,?id22=667879,"О присвоении ISIN кода облигациям АО ""Сбербанк..."
1,28.12.24,?id22=667877,"О присвоении ISIN кода облигациям АО ""Сбербанк..."
2,28.12.24,?id22=667875,"О присвоении ISIN кода облигациям АО ""Сбербанк..."
3,28.12.24,?id22=667873,"О присвоении ISIN кода облигациям АО ""Сбербанк..."
4,28.12.24,?id22=667871,"О присвоении ISIN кода облигациям АО ""Сбербанк..."
...,...,...,...
6030,10.01.22,?id22=651006,О присвоении ISIN кода облигациям Банка ВТБ (П...
6031,10.01.22,?id22=651004,О присвоении ISIN кода облигациям Банка ВТБ (П...
6032,10.01.22,?id22=651002,О присвоении ISIN кода облигациям Банка ВТБ (П...
6033,10.01.22,?id22=651000,О присвоении ISIN кода облигациям Банка ВТБ (П...


In [57]:
filtered.to_csv('isin_bonds_registration.csv')

In [61]:
filtered['date'] = pd.to_datetime(filtered['date'])
filtered['Year'] = filtered['date'].dt.year
filtered['Month'] = filtered['date'].dt.month

C:\Users\lenovo\AppData\Local\Temp\ipykernel_6836\1375586570.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['date'] = pd.to_datetime(filtered['date'])
C:\Users\lenovo\AppData\Local\Temp\ipykernel_6836\1375586570.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['Year'] = filtered['date'].dt.year
C:\Users\lenovo\AppData\Local\Temp\ipykernel_6836\1375586570.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [63]:
filtered = filtered[filtered['Year'] == 2024].reset_index(drop=True)
filtered

,date,web-link,text,Year,Month
0,2024-12-28,?id22=667879,"О присвоении ISIN кода облигациям АО ""Сбербанк...",2024,12
1,2024-12-28,?id22=667877,"О присвоении ISIN кода облигациям АО ""Сбербанк...",2024,12
2,2024-12-28,?id22=667875,"О присвоении ISIN кода облигациям АО ""Сбербанк...",2024,12
3,2024-12-28,?id22=667873,"О присвоении ISIN кода облигациям АО ""Сбербанк...",2024,12
4,2024-12-28,?id22=667871,"О присвоении ISIN кода облигациям АО ""Сбербанк...",2024,12
...,...,...,...,...,...
2641,2024-09-01,?id22=660562,"О присвоении ISIN кода облигациям ПАО ""ТМК"", в...",2024,9
2642,2024-09-01,?id22=660560,"О присвоении ISIN кода облигациям АО ""АЛЬФА-БА...",2024,9
2643,2024-09-01,?id22=660556,О присвоении ISIN кода облигациям Банка ВТБ (П...,2024,9
2644,2024-09-01,?id22=660550,О присвоении ISIN кода облигациям Банка ВТБ (П...,2024,9


In [64]:
filtered.to_csv('isin_bonds_2024_registration.csv')

In [158]:
filtered['web-link'][0]

'?id22=667879'

In [161]:
url_base = 'https://www.isin.ru/ru/ru_isin/news_c/printable.php'
urls = []
for i in range(len(filtered)):
    url_full = f'{url_base}{filtered["web-link"][i]}'
    urls.append(url_full)

In [162]:
len(urls)

2646

In [163]:
tables = []

for url in tqdm(urls):
    r = requests.get(url)
    if r.status_code == 200:
        table = pd.read_html(r.content)[1].set_index('Данные ISIN кода').T
        tables.append(table)
    else:
        print(f'The error with requesting {url}')
        continue

100%|██████████| 2646/2646 [14:23<00:00,  3.06it/s]


In [164]:
len(tables)

2646

In [165]:
df = pd.DataFrame()
for table in tables:
    df = pd.concat([df, table], ignore_index=True, axis=0)

In [166]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2646 entries, 0 to 2645
Data columns (total 21 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   ISIN код                                    2646 non-null   object
 1   Дата присвоения кода                        2646 non-null   object
 2   Параметры ценной бумаги (выпуска)           2635 non-null   object
 3   Эмитент                                     2635 non-null   object
 4   ИНН эмитента                                2635 non-null   object
 5   Наименование выпуска/транша                 2635 non-null   object
 6   Форма выпуска ценной бумаги                 2635 non-null   object
 7   Порядок хранения/учета                      2619 non-null   object
 8   Pегистрационный номер                       2635 non-null   object
 9   Дата регистрации                            2645 non-null   object
 10  Номинальная стоимость ка

In [167]:
df.to_csv('isin_bonds_2024_info_final.csv')

In [170]:
df['Валюта номинала'].unique()

array(['Рубли', 'Дирхам (ОАЭ)', 'Доллар США', 'Евро', 'Юань', nan,
       'Швейцарский франк'], dtype=object)

In [175]:
cny = df[df['Валюта номинала'] == 'Юань']
cny.info()

<class 'pandas.core.frame.DataFrame'>
Index: 72 entries, 388 to 2630
Data columns (total 21 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   ISIN код                                    72 non-null     object
 1   Дата присвоения кода                        72 non-null     object
 2   Параметры ценной бумаги (выпуска)           72 non-null     object
 3   Эмитент                                     72 non-null     object
 4   ИНН эмитента                                72 non-null     object
 5   Наименование выпуска/транша                 72 non-null     object
 6   Форма выпуска ценной бумаги                 72 non-null     object
 7   Порядок хранения/учета                      72 non-null     object
 8   Pегистрационный номер                       72 non-null     object
 9   Дата регистрации                            72 non-null     object
 10  Номинальная стоимость каждой 

In [176]:
cny.to_csv('cny_bonds_2024.csv')